In [160]:
import pandas as pd
import numpy as np
import itertools
import random
import ast
from scipy.spatial import distance
from pyxdameraulevenshtein import damerau_levenshtein_distance

pd.set_option('display.max_rows', 500)

In [162]:
df = pd.read_csv('all_task_environments.csv', header=0)
df.head()

,ID,sequence,objects,coordinates,start_coordinates,strong_k,mid_k,no_k,containment
0,T3,"tn,pc,s","t,n,s,p,c","c: (1,4,4);n: (1,1,2);p: (1,4,4);s: (1,1,2);t:...","[2,1,2],[4,3,2],[4,3,2]",t,"p,n","s,c","p,c,s"
1,T1,tnpsc,"t,n,s,p,c","c: (1,4,4);n: (1,1,2);p: (1,4,4);s: (1,1,2);t:...","[2,1,2];[4,3,2];[4,3,2]",t,"p,n","s,c","p,c,s"
2,T14,tnpcs,"t,n,s,p,c","c: (1,4,4);n: (1,1,2);p: (1,4,4);s: (1,1,2);t:...","[2,1,2];[4,3,2];[4,3,2]",t,"p,n","s,c","p,c,s"
3,T16,tnpsc,"t,n,s,p,c","c: (1,4,4);n: (1,1,2);p: (1,4,4);s: (1,1,2);t:...","[2,1,2];[4,3,2];[4,3,2]",t,"p,n","s,c","p,c,s"
4,T18,tnspc,"t,n,s,p,c","c: (1,4,4);n: (1,1,2);p: (1,4,4);s: (1,1,2);t:...","[2,1,2];[4,3,2];[4,3,2]",t,"p,n","s,c","p,c,s"


In [170]:
def find_best_fit(data):

    results = pd.DataFrame(index=df.index)
    dimensions = [[1,'x'],[1,'y'],[1,'z'],[2,'xy'],[2,'xz'],[2,'yz'],[3,'xyz']]

    for k in range(0,1,0.1):
        k1 = dict()
        for c in range(1,2,0.1):
            c1 = dict()
            for dim in dimensions:
                for row in range(0, len(df)):
                    objects = list(df.at[row,'objects'].split(','))
                    coordinates = {key: value for key, value in (elem.split(': ') for elem in df.at[row,'coordinates'].split(';'))}
                    start_coordinates = list(ast.literal_eval(df.at[row, 'start_coordinates']))
                    sequence = df.at[row,'sequence']
                    
                    #for obj in objects:
                    c1 = {obj: c if obj in df.at[0, 'containment'] else 1.0 for obj in objects}
                
                    edit_dist = get_average(objects, coordinates, start_coordinates, c1, k1, dim, sequence)

#TODO:
# fix input! (better data format?)
# how to define k?
            

In [20]:
def predict_sequence(possible_items, coordinates, c, k, start_coordinates, dimension=[3,]):
    prediction = []
    possible_items = dict.fromkeys(objects, 0) # generate dict from object list
    coord_index = 0
    start_coords = start_coordinates
    coords = coordinates
    
    if dimension[0] == 3: # no changes if 3D
        pass
        
    elif dimension[0] == 2: # 2D: remove obsolete coordinate
        if dimension[1] == 'xy':
            coords = {key: value[:-1] for key, value in coords.items()}
            start_coords = [x[:-1] for x in start_coords]
                
        elif dimension[1] == 'xz':
            #coords = {key: value for key, value in coords.items()}
            for key, value in coords.items():
                new_value = (value[0], value[-1])
                coords[key] = new_value

            for x in range (0, len(start_coords)):
                start_coords[x] = [start_coords[x][0], start_coords[x][-1]]
        
        elif dimension[1] == 'yz':
            coords = {key: value[1:] for key, value in coords.items()}
            start_coords = [x[1:] for x in start_coords]
                
    elif dimension[0] == 1: # 1D: choose appropriate coordinate
        if dimension[1] == 'x':
            coords = {key: value[0] for key, value in coords.items()}
            start_coords = [x[0] for x in start_coords]
        
        elif dimension[1] == 'y':
            coords = {key: value[1] for key, value in coords.items()}
            start_coords = [x[1] for x in start_coords]
        
        elif dimension[1] == 'z':
            coords = {key: value[2] for key, value in coords.items()}
            start_coords = [x[2] for x in start_coords]
    
    while bool(possible_items) == True: # while dict not empty
        for obj in possible_items.keys():
            possible_items[obj] = ((distance.euclidean(
                                start_coords[coord_index], 
                                coords[obj])
                                ) ** k[obj]) * c[obj]
        #print(possible_items)                      
        minval = min(possible_items.values())
        minval = [k for k, v in possible_items.items() if v == minval]
        minval = random.choice(minval) # choose prediction randomly if multiple items have same cost
        prediction.append(minval)
        del possible_items[minval]
        coord_index += 1
        
    return prediction

In [29]:
#predict_sequence(objects,coordinates, c0, k0, start_coordinates, dimensions=[2,'xy'])

['a', 'b', 'c']

In [145]:
def get_average(objects, coordinates, start_coordinates, c, k, dimension, sequence):

    edit_list = []

    for x in range(0,1000):
        result = ''.join(predict_sequence(objects, coordinates, c, k, start_coordinates, dimension))
        #print(result)
        dl = damerau_levenshtein_distance(sequence,result)
        edit_list.append(dl)
    
    avg = np.mean(edit_list)
    return avg

In [129]:
# test format of objects
objects = list(df.at[0,'objects'].split(','))
objects

['t', 'n', 's', 'p', 'c']

In [126]:
# test assignment of c1
c = 1.4
c4 = {obj: c if obj in df.at[0, 'containment'] else 1.0 for obj in objects}
c4

{'t': 1.0, 'n': 1.0, 's': 1.4, 'p': 1.4, 'c': 1.4}

In [133]:
# test format of dict
coords = df.at[0,'coordinates']
d = {key: value for key, value in (elem.split(': ') for elem in coords.split(';'))}
d

{'c': '(1,4,4)',
 'n': '(1,1,2)',
 'p': '(1,4,4)',
 's': '(1,1,2)',
 't': '(1,1,2)',
 'start': '(2,1,2)',
 'table': '(4,3,2)'}

In [172]:
# test format of start_coordinates
x = list(ast.literal_eval(df.at[0,'start_coordinates']))
x

[[2, 1, 2], [4, 3, 2], [4, 3, 2]]